In [1]:
import pandas as pd
import numpy as np
import usaddress
import requests
import os 
# import boto3


## Hunter's Idea (not quite)

In [2]:
def handle_wrong_address(wrong_address):
    """
    Try to handle wrong address, now just use comma to split address and use the part which creates more tag
    :param wrong_address: the addresses cannot be tagged
    :return: tags created
    """
    tag_tests = []
    for wrong_item in wrong_address:
        print(wrong_item)
        updated_item = wrong_item[1].split(',')
        if len(updated_item) != 1:
            new_tags = []
            for part in updated_item:
                try:
                    tag = usaddress.tag(part)
                except:
                    continue
                else:
                    new_tags.append(tag)
            max_len_tag = new_tags[0]
            for new_tag in new_tags[1:]:
                if len(new_tag) > len(max_len_tag):
                    max_len_tag = new_tag
            # print(max_len_tag)
            tag_tests.append(max_len_tag)
    print("Updated tags: {}".format(len(tag_tests)))
    return tag_tests




In [3]:
def addr_tag(location): 
    """
    returns the tag, tag type 
    for location 
    """
    tag = usaddress.tag(location)
    tag_type = tag[1]
    return (tag, tag_type)

In [4]:
if __name__=="__main__":
    input_file = "./first100.csv"
    data = (pd.read_csv(input_file).
           rename(columns=
                  {"Do you have a suggestion or suggestions for a future Slow Street? Please enter it here, in the following format: ABC Street, from X Street to Y Street"
                   :'Location'}))
    missing_location_field = []
    tagged_address = []
    tagged_intersect = []
    tagged_ambiguous = []
    repeated_label_error = []
    clean_prep = []
    for row in data.iterrows():
        # print("Parsing {}".format(row))
        row = row[1] ## eliminate index/series crap. 
        location = row.Location
        if type(location) == float:
            missing_location_field.append(row)
        elif len(location) == 0 or location == 'None' or location == 'Not stated' or location == 'nan':
            missing_location_field.append(row)
        else: 
            try: 
                tag = usaddress.tag(location)
            except usaddress.RepeatedLabelError as e:
                repeated_label_error.append(location)
                continue
            tag_type = tag[1]
            if tag_type =='Ambiguous':
                tagged_ambiguous.append(row)
            elif tag_type == 'Street Address': 
                tagged_address.append(row)
            elif tag_type == 'Intersection':
                tagged_intersect.append(row)
            clean_prep.append(row)
    df = pd.DataFrame(clean_prep)
    df['tag_tuple'] = df['Location'].apply(addr_tag)
    df['tag'] = df['tag_tuple'].apply(lambda x: x[0])
    df['tag_type'] = df['tag_tuple'].apply(lambda x: x[1])
    del df['tag_tuple']
    print("""Overall Parsing Step Stats: Total missing loc: {}
                                         Total address: {} 
                                         Total ambiguous: {} 
                                         Total intersection: {} 
                                         Total repeated label {}
          """.format(len(missing_location_field),
                     len(tagged_address),
                     len(tagged_ambiguous),
                     len(tagged_intersect),
                     len(repeated_label_error)))
    missing_locations_fp = "./processed/missing_locations.csv"
    pd.DataFrame(missing_location_field).to_csv(missing_locations_fp)
    print("wrote missing locations to {}".format(missing_locations_fp))
    repeated_locations_fp = "./processed/repeated_locations.csv"
    pd.DataFrame(repeated_label_error).to_csv(repeated_locations_fp) 
    print("wrote repeated locations to {}".format(repeated_locations_fp))
    feather_loc = "./processed/df_with_tags.p"
    pd.DataFrame(df).to_pickle(feather_loc)
    print("wrote tags to pickle at {}".format(feather_loc))

Overall Parsing Step Stats: Total missing loc: 18
                                         Total address: 6 
                                         Total ambiguous: 73 
                                         Total intersection: 0 
                                         Total repeated label 1
          
wrote missing locations to ./processed/missing_locations.csv
wrote repeated locations to ./processed/repeated_locations.csv
wrote tags to pickle at ./processed/df_with_tags.p


In [5]:
results = pd.read_pickle('./processed/df_with_tags.p')

In [6]:
results

Response ID Language  Postal  \
0             3  English   94123   
1             4  English   94110   
2             5  English   94122   
3             6  English   94080   
4             7  English   94606   
..          ...      ...     ...   
92           95  English   94115   
93           96  English   94117   
94           97  English   94116   
95           98  English   94110   
97          100  English   94116   

                                             Location  \
0   6th Street/Taylor, from Mission to O'Farrell; ...   
1         Folsom Street, any number of SoMa alleyways   
2   JFK Drive in GG Park. People should not be dri...   
3   Terry A Francois Blvd. (between Mission Rock a...   
4   Powhattan:: bocaba/ wool to Andersen   Would s...   
..                                                ...   
92  Divisadero Street, from Marina Blvd to 14th St...   
93                                                JFK   
94                Embarcadero, from 2nd to Bay street   
95  24th Street, from South Van Ness Ave to Potrer...   
97  JFK Drive from Stanyan St to Great Highway 23r...   

    What is your home zip code? P/N/O  \
0                       94102.0     P   
1                       94103.0     P   
2                       94122.0     P   
3                       94158.0     P   
4                       94110.0     P   
..                          ...   ...   
92                      94115.0     P   
93                      94114.0     P   
94                      94131.0     P   
95                      94110.0     P   
97                      94131.0     P   

                                                  tag        tag_type  
0   ({'Recipient': '6th Street/Taylor, from Missio...       Ambiguous  
1   ({'Recipient': 'Folsom Street, any number of S...       Ambiguous  
2   ({'BuildingName': 'JFK Drive in GG Park. Peopl...       Ambiguous  
3   ({'Recipient': 'Terry A Francois Blvd. (betwee...       Ambiguous  
4   ({'Recipient': 'Powhattan:: bocaba/ wool to An...       Ambiguous  
..                                                ...             ...  
92  ({'StreetName': 'Divisadero', 'StreetNamePostT...       Ambiguous  
93                ({'CountryName': 'JFK'}, Ambiguous)       Ambiguous  
94  ({'Recipient': 'Embarcadero, from', 'AddressNu...  Street Address  
95  ({'StreetName': '24th', 'StreetNamePostType': ...       Ambiguous  
97  ({'Recipient': 'JFK Drive from Stanyan St to G...       Ambiguous  

[79 rows x 8 columns]

In [7]:
    # handle_result = handle_wrong_address(wrong_address)
    # clean_data_with_google(locations)

## Me Trying Stuff

In [8]:
loc_str = "6th Street/Taylor, from Mission to O'Farrell; 7th Street/Jones, from Mission to Turk 7th Street/Leavenworth, from Mission to O'Farrell 8th Street/Hyde, from Mission to O'Farrell"

In [9]:
loc_str

"6th Street/Taylor, from Mission to O'Farrell; 7th Street/Jones, from Mission to Turk 7th Street/Leavenworth, from Mission to O'Farrell 8th Street/Hyde, from Mission to O'Farrell"

In [10]:
ls2 = "Greenwich, from Polk to Lyon Chestnut, from Polk to Lyon Columbus, from Beach to Montgomery North Point, from Van Ness to Embarcadero"

In [11]:
ls2

'Greenwich, from Polk to Lyon Chestnut, from Polk to Lyon Columbus, from Beach to Montgomery North Point, from Van Ness to Embarcadero'

In [12]:
ls2_words = ls2.split(" ")

In [13]:
sf_streets = pd.read_csv('./raw/SF_Street_Names.csv')

In [14]:
sf_st_list = list(sf_streets['StreetName'])

#remove leading 0
sf_st_list = [st[1:] if st[0] == '0' else st for st in sf_st_list]
#match case
# sf_st_list = [st[0] + st[1:].lower() for st in sf_st_list]
# sf_st_list = [st.title() if not st[0].isnumeric() else st.lower() for st in sf_st_list]
sf_st_list = [st.lower() for st in sf_st_list]

In [15]:
ls2

'Greenwich, from Polk to Lyon Chestnut, from Polk to Lyon Columbus, from Beach to Montgomery North Point, from Van Ness to Embarcadero'

In [16]:
import difflib

In [17]:
## from https://stackoverflow.com/questions/41192424/python-how-to-correct-misspelled-names, not entirely ideal

In [18]:
def is_similar(first, second, ratio):
    return difflib.SequenceMatcher(None, first, second).ratio() > ratio

In [19]:
result = [s for f in ls2_words for s in sf_st_list if is_similar(f,s, 0.9)]

In [20]:
result

[]

In [21]:
import spacy

In [22]:
nlp = spacy.load("en_core_web_sm")
# doc = nlp(ls2)
doc = nlp("o'farrrel, from pok to lyon")
propns = []
for token in doc:
    print(token.text, token.pos_, token.dep_)
    if token.pos_ == 'PROPN':
        propns += [token.text]

o'farrrel PROPN ROOT
, PUNCT punct
from ADP prep
pok PROPN pobj
to ADP prep
lyon PROPN pobj


In [23]:
propns

["o'farrrel", 'pok', 'lyon']

In [24]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

o'farrrel 0 9 PERSON
pok 16 19 ORG


### Eventually include misspellings something like this?

In [25]:
first100 = (pd.read_csv(input_file).
       rename(columns=
              {"Do you have a suggestion or suggestions for a future Slow Street? Please enter it here, in the following format: ABC Street, from X Street to Y Street"
               :'Location'}))

In [26]:
all_text = ' '.join(list(first100['Location'].astype(str)))

In [27]:
all_split = all_text.split(' ')

In [28]:
all_text +=  "Aleminy albyon Albyon alen Alen o'farrell o'farrel"

In [29]:
nlp = spacy.load("en_core_web_sm")
# doc = nlp(ls2)
doc = nlp(all_text)
propns = []
for token in doc:
#     print(token.text, token.pos_, token.dep_)
    if token.pos_ == 'PROPN':
        propns += [token.text]

In [30]:
'Aleminy' in propns

False

In [31]:
propns = [n.lower() for n in propns]

In [32]:
all_matches = {}
for street in sf_st_list:
    st_matches = difflib.get_close_matches(street, propns, n=3, cutoff = 0.8)
    if len(list(set(st_matches))) > 0:
        all_matches[street] = list(set(st_matches))

In [33]:
all_matches['ofarrell']
# get all values
# add them to sf streets, then set for unique and nix "avenue, st" before rule-making step...

["o'farrell"]

In [34]:
all_matches

{'1st': ['st'],
 '2nd': ['2nd'],
 '3rd': ['3rd'],
 '4th': ['24th', '4th'],
 '6th': ['6th'],
 '7th': ['7th'],
 '8th': ['8th'],
 '13th': ['13th'],
 '14th': ['4th'],
 '16th': ['6th'],
 '17th': ['7th'],
 '18th': ['8th'],
 '22nd': ['2nd'],
 '23rd': ['3rd'],
 '24th': ['24th', '4th'],
 '26th': ['6th'],
 '27th': ['7th'],
 '28th': ['8th'],
 '32nd': ['2nd'],
 '33rd': ['3rd'],
 '34th': ['4th'],
 '36th': ['6th'],
 '37th': ['7th'],
 '38th': ['8th'],
 '40th': ['4th'],
 '42nd': ['2nd'],
 '43rd': ['3rd'],
 '44th': ['4th'],
 '45th': ['4th'],
 '46th': ['4th', '6th'],
 '47th': ['7th', '4th'],
 '48th': ['8th', '4th'],
 'a': ['a'],
 'addison': ['madison'],
 'agua': ['laguna'],
 'albion': ['albyon'],
 'alemany': ['alemany'],
 'allen': ['alen'],
 'alpine': ['pine'],
 'anderson': ['andersen'],
 'anza': ['anza'],
 'arguello': ['arguello'],
 'athens': ['athens'],
 'avalon': ['avalon'],
 'avenue b': ['avenue'],
 'avenue c': ['avenue'],
 'avenue d': ['avenue'],
 'avenue e': ['avenue'],
 'avenue f': ['avenue'],
 '

In [35]:
list(all_matches.values())

[['st'],
 ['2nd'],
 ['3rd'],
 ['24th', '4th'],
 ['6th'],
 ['7th'],
 ['8th'],
 ['13th'],
 ['4th'],
 ['6th'],
 ['7th'],
 ['8th'],
 ['2nd'],
 ['3rd'],
 ['24th', '4th'],
 ['6th'],
 ['7th'],
 ['8th'],
 ['2nd'],
 ['3rd'],
 ['4th'],
 ['6th'],
 ['7th'],
 ['8th'],
 ['4th'],
 ['2nd'],
 ['3rd'],
 ['4th'],
 ['4th'],
 ['4th', '6th'],
 ['7th', '4th'],
 ['8th', '4th'],
 ['a'],
 ['madison'],
 ['laguna'],
 ['albyon'],
 ['alemany'],
 ['alen'],
 ['pine'],
 ['andersen'],
 ['anza'],
 ['arguello'],
 ['athens'],
 ['avalon'],
 ['avenue'],
 ['avenue'],
 ['avenue'],
 ['avenue'],
 ['avenue'],
 ['avenue'],
 ['avenue'],
 ['avenue'],
 ['avenue'],
 ['avalon'],
 ['beach'],
 ['baker'],
 ['balboa'],
 ['bay'],
 ['bayshore'],
 ['beach'],
 ['bernal'],
 ['ferry', 'terry'],
 ['lake'],
 ['bryant'],
 ['green'],
 ['broadway'],
 ['brotherhood'],
 ['bush'],
 ['bryant'],
 ['burnett'],
 ['bush'],
 ['cabrillo'],
 ['california'],
 ['capp'],
 ['carmel'],
 ['carmel'],
 ['carmel'],
 ['carroll'],
 ['caesar'],
 ['castro'],
 ['cayuga'],
 

In [36]:
sf_st_list[1600:]

['miller',
 'miller',
 'millwright cottage access',
 'milton',
 'milton i ross',
 'minerva',
 'minna',
 'minnesota',
 'mint',
 'mint',
 'mirabel',
 'miraloma',
 'miramar',
 'mirando',
 'mission',
 'mission bay',
 'mission bay',
 'mission rock',
 'mississippi',
 'missouri',
 'mistral',
 'mizpah',
 'modoc',
 'moffitt',
 'mojave',
 'molimo',
 'moncada',
 'moneta',
 'moneta',
 'mono',
 'montague',
 'montalvo',
 'montana',
 'montcalm',
 'park',
 'montclair',
 'monte vista',
 'montecito',
 'monterey',
 'montezuma',
 'montgomery',
 'monticello',
 'monument',
 'moore',
 'moore',
 'moraga',
 'moraga',
 'moreland',
 'morgan',
 'morningside',
 'morrell',
 'morrell',
 'morris',
 'morse',
 'morton',
 'moscow',
 'moss',
 'moulton',
 'moultrie',
 'mount',
 'mount vernon',
 'mountain spring',
 'mountview',
 'muir',
 'muir',
 'mulford',
 'mullen',
 'munich',
 'murray',
 'murray',
 'museum',
 'music concourse',
 'music concourse access',
 'myra',
 'myrtle',
 'nadell',
 'naglee',
 'nahua',
 'nancy pelosi

### Find SF Streets!!

In [37]:
##Add common exceptions
sf_st_list += ['embarcadero', "o'farrell"]

In [38]:
patterns = []
for street in sf_st_list:
    pattern = []
    for word in street.split(" "):
        pattern += [{'LOWER': word}]
    patterns += [pattern]

In [39]:
patterns[1500:]

[[{'LOWER': 'marcela'}],
 [{'LOWER': 'marcy'}],
 [{'LOWER': 'marengo'}],
 [{'LOWER': 'margaret'}],
 [{'LOWER': 'margrave'}],
 [{'LOWER': 'marietta'}],
 [{'LOWER': 'marin'}],
 [{'LOWER': 'marina'}],
 [{'LOWER': 'marina'}, {'LOWER': 'green'}],
 [{'LOWER': 'marine'}],
 [{'LOWER': 'mariner'}],
 [{'LOWER': 'marion'}],
 [{'LOWER': 'mariposa'}],
 [{'LOWER': 'marist'}],
 [{'LOWER': 'mark'}],
 [{'LOWER': 'mark'}, {'LOWER': 'twain'}],
 [{'LOWER': 'market'}],
 [{'LOWER': 'marlin'}],
 [{'LOWER': 'marne'}],
 [{'LOWER': 'mars'}],
 [{'LOWER': 'marshall'}],
 [{'LOWER': 'marsily'}],
 [{'LOWER': 'marston'}],
 [{'LOWER': 'martha'}],
 [{'LOWER': 'martin'},
  {'LOWER': 'luther'},
  {'LOWER': 'king'},
  {'LOWER': 'jr'}],
 [{'LOWER': 'martinez'}],
 [{'LOWER': 'marvel'}],
 [{'LOWER': 'marview'}],
 [{'LOWER': 'mary'}],
 [{'LOWER': 'mary'}, {'LOWER': 'teresa'}],
 [{'LOWER': 'maryland'}],
 [{'LOWER': 'mason'}],
 [{'LOWER': 'mason'}],
 [{'LOWER': 'masonic'}],
 [{'LOWER': 'massachusetts'}],
 [{'LOWER': 'massasoit'

In [40]:
##Add common exceptions
patterns += [[{'LOWER': 'embarcadero'}]]

In [41]:
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab, validate=True)
# Add match ID "HelloWorld" with no callback and one pattern
# pattern = [{"LOWER": "north"}, {"LOWER": "point"}]
# matcher.add("SFstreet", None, pattern)

for pattern in patterns:
    matcher.add("SFstreet", None, pattern)



In [42]:
def old_matcher(doc):
    matches = matcher(doc)
    starts = []
    streets = []
    for match_id, start, end in matches:
        # Avoid the North and North Point bug...
        ##TODO fix the Martin Luther King bug... (double repitition...)
        if start in starts:
            streets = streets[:-1]
        starts += [start]
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        print(match_id, string_id, start, end, span.text)
        streets += [span.text]
    return streets

In [43]:
def test_matcher(doc):
    matches = matcher(doc)
    starts = []
    streets = {}
    keys_to_ditch = []
    for match_id, start, end in matches:
        # Avoid the North and North Point bug...
        ##TODO fix the Martin Luther King bug... (double repitition...)
        for key in streets.keys():
#             print('key: ', key)
            if np.arange(key[0], key[1]) in np.arange(start,end):
#                 print('pop')
                keys_to_ditch.append(key)
#                 streets.pop(key, None)
                
        starts += [start]
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        print(match_id, string_id, start, end, span.text)
        streets[(start,end)] = span.text
        
        for key in keys_to_ditch:
            streets.pop(key, None)
#     print(list(streets.values()))
    return list(streets.values())

In [44]:
ls2

'Greenwich, from Polk to Lyon Chestnut, from Polk to Lyon Columbus, from Beach to Montgomery North Point, from Van Ness to Embarcadero'

In [45]:
loc_str

"6th Street/Taylor, from Mission to O'Farrell; 7th Street/Jones, from Mission to Turk 7th Street/Leavenworth, from Mission to O'Farrell 8th Street/Hyde, from Mission to O'Farrell"

In [46]:
doc = nlp(ls2)
streets = test_matcher(doc)

12328089489653534324 SFstreet 0 1 Greenwich
12328089489653534324 SFstreet 3 4 Polk
12328089489653534324 SFstreet 5 6 Lyon
12328089489653534324 SFstreet 6 7 Chestnut
12328089489653534324 SFstreet 9 10 Polk
12328089489653534324 SFstreet 11 12 Lyon
12328089489653534324 SFstreet 12 13 Columbus
12328089489653534324 SFstreet 15 16 Beach
12328089489653534324 SFstreet 17 18 Montgomery
12328089489653534324 SFstreet 18 19 North
12328089489653534324 SFstreet 18 20 North Point
12328089489653534324 SFstreet 22 24 Van Ness
12328089489653534324 SFstreet 25 26 Embarcadero


In [47]:
streets

['Greenwich',
 'Polk',
 'Lyon',
 'Chestnut',
 'Polk',
 'Lyon',
 'Columbus',
 'Beach',
 'Montgomery',
 'North Point',
 'Van Ness',
 'Embarcadero']

In [48]:
segments = [['6th', 'Taylor', 'Mission'], ["O'Farrell", '7th', 'Jones'], ['Mission', 'Turk', '7th'], ['Leavenworth', 'Mission', "O'Farrell"], ['8th', 'Hyde', 'Mission'], ["O'Farrell"]]

In [50]:
# [[segment[0] + '/' + segment[1]] + [segment[0] + '/' + segment[2]] for segment in segments]

In [51]:
n = 3
segments = [streets[i * n:(i + 1) * n] for i in range((len(streets) + n - 1) // n )]

[[segment[0] + '/' + segment[1]] + [segment[0] + '/' + segment[2]] for segment in segments]

[['Greenwich/Polk', 'Greenwich/Lyon'],
 ['Chestnut/Polk', 'Chestnut/Lyon'],
 ['Columbus/Beach', 'Columbus/Montgomery'],
 ['North Point/Van Ness', 'North Point/Embarcadero']]

In [54]:
## hell yeah, this list works for the in-threes method!!!

In [55]:
locations = first100.dropna(subset=['Location']).reset_index()['Location']

In [56]:
mlk = locations[locations.str.contains('Luther')]

In [57]:
mlk[58]

'Martin Luther King Jr Dr, from Lincoln Way to Lincoln Way'

In [58]:
def segments_from_location(location_str):
    doc = nlp(location_str)
    streets = test_matcher(doc)
    
    n = 3
    segments = [streets[i * n:(i + 1) * n] for i in range((len(streets) + n - 1) // n )]
#     print(segments)
    #drop mismatched segments... (improve?)
    segments = [segment for segment in segments if len(segment) == 3]
    x_street_sep = ' and '
    global _debug
    _debug = segments
    segments = [[segment[0] + x_street_sep + segment[1]] + [segment[0] + x_street_sep + segment[2]] for segment in segments]
    return segments

In [59]:
def add_df_segments(df):
    df['segments'] = segments_from_location(df['Location'])
    return df

In [60]:
loc_segments = locations.apply(segments_from_location)

12328089489653534324 SFstreet 0 1 6th
12328089489653534324 SFstreet 3 4 Taylor
12328089489653534324 SFstreet 6 7 Mission
12328089489653534324 SFstreet 8 9 O'Farrell
12328089489653534324 SFstreet 10 11 7th
12328089489653534324 SFstreet 13 14 Jones
12328089489653534324 SFstreet 16 17 Mission
12328089489653534324 SFstreet 18 19 Turk
12328089489653534324 SFstreet 19 20 7th
12328089489653534324 SFstreet 22 23 Leavenworth
12328089489653534324 SFstreet 25 26 Mission
12328089489653534324 SFstreet 27 28 O'Farrell
12328089489653534324 SFstreet 28 29 8th
12328089489653534324 SFstreet 31 32 Hyde
12328089489653534324 SFstreet 34 35 Mission
12328089489653534324 SFstreet 36 37 O'Farrell
12328089489653534324 SFstreet 0 1 Folsom
12328089489653534324 SFstreet 4 5 Park
12328089489653534324 SFstreet 13 14 park
12328089489653534324 SFstreet 24 25 a
12328089489653534324 SFstreet 51 52 lane
12328089489653534324 SFstreet 1 2 A
12328089489653534324 SFstreet 0 3 Terry A Francois
12328089489653534324 SFstreet 7 

<ipython-input-43-d4df93aa8819>:11: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.arange(key[0], key[1]) in np.arange(start,end):


12328089489653534324 SFstreet 0 1 Haight
12328089489653534324 SFstreet 3 4 Octavia
12328089489653534324 SFstreet 5 6 Stanyan
12328089489653534324 SFstreet 8 9 Stanyan
12328089489653534324 SFstreet 11 12 Great
12328089489653534324 SFstreet 13 14 Cole
12328089489653534324 SFstreet 16 17 Haight
12328089489653534324 SFstreet 18 19 Carmel
12328089489653534324 SFstreet 5 6 Anza
12328089489653534324 SFstreet 7 8 Park
12328089489653534324 SFstreet 7 9 Park Presidio
12328089489653534324 SFstreet 8 9 Presidio
12328089489653534324 SFstreet 10 11 25th
12328089489653534324 SFstreet 12 13 Clement
12328089489653534324 SFstreet 16 17 Park
12328089489653534324 SFstreet 16 18 Park Presidio
12328089489653534324 SFstreet 17 18 Presidio
12328089489653534324 SFstreet 19 20 Balboa
12328089489653534324 SFstreet 21 22 32nd
12328089489653534324 SFstreet 23 24 45th
12328089489653534324 SFstreet 24 25 Lake
12328089489653534324 SFstreet 26 27 Park
12328089489653534324 SFstreet 26 28 Park Presidio
12328089489653534

12328089489653534324 SFstreet 0 1 Leavenworth
12328089489653534324 SFstreet 4 5 Ellis
12328089489653534324 SFstreet 7 8 McAllister
12328089489653534324 SFstreet 9 10 California
12328089489653534324 SFstreet 13 14 Hyde
12328089489653534324 SFstreet 16 17 Franklin
12328089489653534324 SFstreet 18 19 4th
12328089489653534324 SFstreet 22 23 Harrison
12328089489653534324 SFstreet 25 26 Townsend
12328089489653534324 SFstreet 28 29 east
12328089489653534324 SFstreet 31 33 The Embarcadero
12328089489653534324 SFstreet 32 33 Embarcadero
12328089489653534324 SFstreet 35 36 Townsend
12328089489653534324 SFstreet 38 39 North
12328089489653534324 SFstreet 38 40 North Point
12328089489653534324 SFstreet 42 43 east
12328089489653534324 SFstreet 44 45 water
12328089489653534324 SFstreet 55 56 west
12328089489653534324 SFstreet 0 1 Paris
12328089489653534324 SFstreet 3 4 Avalon
12328089489653534324 SFstreet 5 6 Geneva
12328089489653534324 SFstreet 7 8 Madrid
12328089489653534324 SFstreet 10 11 Silver
1

12328089489653534324 SFstreet 1 2 Stockton
12328089489653534324 SFstreet 3 4 union
12328089489653534324 SFstreet 6 7 a
12328089489653534324 SFstreet 9 10 green
12328089489653534324 SFstreet 18 19 I
12328089489653534324 SFstreet 34 35 Powell
12328089489653534324 SFstreet 3 4 Stanyan
12328089489653534324 SFstreet 6 7 Great
12328089489653534324 SFstreet 0 1 Divisadero
12328089489653534324 SFstreet 4 5 Marina
12328089489653534324 SFstreet 7 8 14th
12328089489653534324 SFstreet 9 10 Kezar
12328089489653534324 SFstreet 13 14 Lincoln
12328089489653534324 SFstreet 16 17 Stanyan
12328089489653534324 SFstreet 18 19 Oak
12328089489653534324 SFstreet 22 23 Stanyan
12328089489653534324 SFstreet 25 27 Van Ness
12328089489653534324 SFstreet 28 29 Fell
12328089489653534324 SFstreet 32 33 Stanyan
12328089489653534324 SFstreet 35 37 Van Ness
12328089489653534324 SFstreet 38 39 Fulton
12328089489653534324 SFstreet 42 43 Great
12328089489653534324 SFstreet 45 46 Franklin
12328089489653534324 SFstreet 47 4

In [61]:
for segment in loc_segments:
    print(segment)

[['6th and Taylor', '6th and Mission'], ["O'Farrell and 7th", "O'Farrell and Jones"], ['Mission and Turk', 'Mission and 7th'], ['Leavenworth and Mission', "Leavenworth and O'Farrell"], ['8th and Hyde', '8th and Mission']]
[]
[['Park and park', 'Park and a']]
[['Terry A Francois and Rock', 'Terry A Francois and Mariposa']]
[['Powhattan and wool', 'Powhattan and still'], ['north and south', 'north and bernal Heights']]
[]
[['Embarcadero and King', 'Embarcadero and Embarcadero']]
[['Haight and Octavia', 'Haight and Stanyan'], ['Stanyan and Great', 'Stanyan and Cole']]
[['Anza and Presidio', 'Anza and 25th'], ['Clement and Presidio', 'Clement and Balboa'], ['32nd and 45th', '32nd and Lake']]
[['Dolores and San Jose', 'Dolores and Market']]
[['Grant and Bush', 'Grant and Columbus'], ['Water and Embarcadero', 'Water and Harrison'], ['Powell and Taylor', 'Powell and California']]
[['Hampshire and 24th', 'Hampshire and York'], ['24th and 25th', '24th and Potrero'], ['Bryant and 26th', 'Bryant 

In [62]:
#wicked cool, now go to bed boy.

In [64]:
first100_segmented = first100.dropna(subset=['Location']).apply(add_df_segments, axis=1)

12328089489653534324 SFstreet 0 1 6th
12328089489653534324 SFstreet 3 4 Taylor
12328089489653534324 SFstreet 6 7 Mission
12328089489653534324 SFstreet 8 9 O'Farrell
12328089489653534324 SFstreet 10 11 7th
12328089489653534324 SFstreet 13 14 Jones
12328089489653534324 SFstreet 16 17 Mission
12328089489653534324 SFstreet 18 19 Turk
12328089489653534324 SFstreet 19 20 7th
12328089489653534324 SFstreet 22 23 Leavenworth
12328089489653534324 SFstreet 25 26 Mission
12328089489653534324 SFstreet 27 28 O'Farrell
12328089489653534324 SFstreet 28 29 8th
12328089489653534324 SFstreet 31 32 Hyde
12328089489653534324 SFstreet 34 35 Mission
12328089489653534324 SFstreet 36 37 O'Farrell
12328089489653534324 SFstreet 0 1 Folsom
12328089489653534324 SFstreet 4 5 Park
12328089489653534324 SFstreet 13 14 park
12328089489653534324 SFstreet 24 25 a
12328089489653534324 SFstreet 51 52 lane
12328089489653534324 SFstreet 1 2 A
12328089489653534324 SFstreet 0 3 Terry A Francois
12328089489653534324 SFstreet 7 

<ipython-input-43-d4df93aa8819>:11: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.arange(key[0], key[1]) in np.arange(start,end):


12328089489653534324 SFstreet 5 6 Anza
12328089489653534324 SFstreet 7 8 Park
12328089489653534324 SFstreet 7 9 Park Presidio
12328089489653534324 SFstreet 8 9 Presidio
12328089489653534324 SFstreet 10 11 25th
12328089489653534324 SFstreet 12 13 Clement
12328089489653534324 SFstreet 16 17 Park
12328089489653534324 SFstreet 16 18 Park Presidio
12328089489653534324 SFstreet 17 18 Presidio
12328089489653534324 SFstreet 19 20 Balboa
12328089489653534324 SFstreet 21 22 32nd
12328089489653534324 SFstreet 23 24 45th
12328089489653534324 SFstreet 24 25 Lake
12328089489653534324 SFstreet 26 27 Park
12328089489653534324 SFstreet 26 28 Park Presidio
12328089489653534324 SFstreet 27 28 Presidio
12328089489653534324 SFstreet 0 1 Dolores
12328089489653534324 SFstreet 2 4 San Jose
12328089489653534324 SFstreet 5 6 Market
12328089489653534324 SFstreet 0 1 Grant
12328089489653534324 SFstreet 3 4 Bush
12328089489653534324 SFstreet 5 6 Columbus
12328089489653534324 SFstreet 8 9 Water
12328089489653534324

12328089489653534324 SFstreet 4 5 mission
12328089489653534324 SFstreet 6 7 17th
12328089489653534324 SFstreet 8 9 Potrero
12328089489653534324 SFstreet 9 10 hill
12328089489653534324 SFstreet 16 17 3rd
12328089489653534324 SFstreet 21 22 bay
12328089489653534324 SFstreet 21 23 bay view
12328089489653534324 SFstreet 33 34 a
12328089489653534324 SFstreet 35 36 lane
12328089489653534324 SFstreet 45 47 The Embarcadero
12328089489653534324 SFstreet 46 47 Embarcadero
12328089489653534324 SFstreet 0 1 24th
12328089489653534324 SFstreet 3 4 Potrero
12328089489653534324 SFstreet 5 6 Mission
12328089489653534324 SFstreet 7 8 York
12328089489653534324 SFstreet 9 10 26th
12328089489653534324 SFstreet 11 12 20th
12328089489653534324 SFstreet 0 1 Grove
12328089489653534324 SFstreet 3 4 Stanyan
12328089489653534324 SFstreet 5 6 Scott
12328089489653534324 SFstreet 6 7 Lake
12328089489653534324 SFstreet 9 13 El Camino del Mar
12328089489653534324 SFstreet 14 15 Arguello
12328089489653534324 SFstreet 1

In [65]:
first100_segmented

Response ID Language  Postal  \
0             3  English   94123   
1             4  English   94110   
2             5  English   94122   
3             6  English   94080   
4             7  English   94606   
..          ...      ...     ...   
92           95  English   94115   
93           96  English   94117   
94           97  English   94116   
95           98  English   94110   
97          100  English   94116   

                                             Location  \
0   6th Street/Taylor, from Mission to O'Farrell; ...   
1         Folsom Street, any number of SoMa alleyways   
2   JFK Drive in GG Park. People should not be dri...   
3   Terry A Francois Blvd. (between Mission Rock a...   
4   Powhattan:: bocaba/ wool to Andersen   Would s...   
..                                                ...   
92  Divisadero Street, from Marina Blvd to 14th St...   
93                                                JFK   
94                Embarcadero, from 2nd to Bay street   
95  24th Street, from South Van Ness Ave to Potrer...   
97  JFK Drive from Stanyan St to Great Highway 23r...   

    What is your home zip code? P/N/O  \
0                       94102.0     P   
1                       94103.0     P   
2                       94122.0     P   
3                       94158.0     P   
4                       94110.0     P   
..                          ...   ...   
92                      94115.0     P   
93                      94114.0     P   
94                      94131.0     P   
95                      94110.0     P   
97                      94131.0     P   

                                             segments  
0   [[6th and Taylor, 6th and Mission], [O'Farrell...  
1                                                  []  
2                       [[Park and park, Park and a]]  
3   [[Terry A Francois and Rock, Terry A Francois ...  
4   [[Powhattan and wool, Powhattan and still], [n...  
..                                                ...  
92  [[Divisadero and Marina, Divisadero and 14th],...  
93                                                 []  
94       [[Embarcadero and 2nd, Embarcadero and Bay]]  
95     [[24th and South Van Ness, 24th and Van Ness]]  
97  [[Stanyan and Great, Stanyan and 23rd], [Dolor...  

[81 rows x 7 columns]